In [1]:
import os


DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")
#SUBINDO UM NIVEL NOS CAMINHOOS

In [2]:
# Bloco Extra: DataLoader Customizado (Mesmo Padrão do U-Net e U-Net++)

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Classe customizada do Dataset (mesmo padrão do U-Net e U-Net++)
class ISICDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = sorted(os.listdir(image_dir))
        self.mask_list = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        # Garante que o índice não vai além do tamanho do dataset
        idx = idx % len(self.image_list)

        # Caminhos das imagens e máscaras
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_name = self.image_list[idx].replace('.jpg', '_segmentation.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Verificação de Existência
        if not os.path.exists(img_path) or not os.path.exists(mask_path):
            print(f"⚠️ Arquivo não encontrado: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Carregar Imagem e Máscara
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Verificação de Corrupção
        if image is None or mask is None:
            print(f"⚠️ Imagem ou Máscara corrompida em: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Conversão e Redimensionamento
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalização
        image = image / 255.0
        mask = (mask > 0).astype(np.float32)  # Converte a máscara para binário (0 ou 1)

        # Conversão para Tensor
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask

# Caminhos relativos (mantendo o padrão)
DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")

# Criar dataset e DataLoader
batch_size = 8

train_dataset = ISICDataset(IMAGE_DIR_TRAIN, MASK_DIR_TRAIN)
val_dataset = ISICDataset(IMAGE_DIR_VAL, MASK_DIR_VAL)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# Teste: Exibir um batch do DataLoader
batch = next(iter(train_loader))
images, masks = batch

print(f"📊 Formato do batch de imagens: {images.shape}")  # Esperado: [8, 3, 256, 256]
print(f"📊 Formato do batch de máscaras: {masks.shape}")  # Esperado: [8, 1, 256, 256]


📊 Formato do batch de imagens: torch.Size([8, 3, 256, 256])
📊 Formato do batch de máscaras: torch.Size([8, 1, 256, 256])


In [3]:
# Bloco 3: Definição do ResUNet

import torch
import torch.nn as nn
import torch.nn.functional as F

# Bloco Residual
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Atalho (skip connection) para correspondência de dimensão
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Definição do ResUNet
class ResUNet(nn.Module):
    def __init__(self, num_classes=1):
        super(ResUNet, self).__init__()

        # Encoder (Downsampling)
        self.encoder1 = ResidualBlock(3, 64)
        self.encoder2 = ResidualBlock(64, 128, stride=2)
        self.encoder3 = ResidualBlock(128, 256, stride=2)
        self.encoder4 = ResidualBlock(256, 512, stride=2)

        # Bridge
        self.bridge = ResidualBlock(512, 1024, stride=2)

        # Decoder (Upsampling)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = ResidualBlock(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = ResidualBlock(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = ResidualBlock(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = ResidualBlock(128, 64)

        # Saída
        self.final_conv = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        # Bridge
        b = self.bridge(e4)

        # Decoder com Skip Connections
        d4 = self.upconv4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.decoder1(d1)

        out = self.final_conv(d1)
        return out

# Instanciando o modelo
num_classes = 1
model = ResUNet(num_classes=num_classes)

# Imprimindo a arquitetura do modelo
print(model)


ResUNet(
  (encoder1): ResidualBlock(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential(
      (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (encoder2): ResidualBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [4]:
# Bloco 4: Funções de Perda e Métricas (Mesmo Padrão do U-Net e U-Net++)

import torch
import torch.nn.functional as F
import torch.optim as optim

# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")
model.to(device)

# Função de perda (Binary Cross Entropy com Dice Loss)
def dice_loss(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Converte logits para probabilidade
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

criterion = lambda pred, target: 0.5 * F.binary_cross_entropy_with_logits(pred, target) + 0.5 * dice_loss(pred, target)

# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Funções para métricas
def iou(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-5) / (union + 1e-5)

def dice_coefficient(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    return (2. * intersection + 1e-5) / (pred.sum() + target.sum() + 1e-5)

def precision(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    predicted_positive = pred.sum()
    return (true_positive + 1e-5) / (predicted_positive + 1e-5)

def recall(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    actual_positive = target.sum()
    return (true_positive + 1e-5) / (actual_positive + 1e-5)

def f1_score(pred, target, threshold=0.5):
    p = precision(pred, target, threshold)
    r = recall(pred, target, threshold)
    return 2 * (p * r) / (p + r + 1e-5)


In [5]:
# Bloco 5: Função de Treinamento e Validação (Mesmo Padrão do U-Net e U-Net++)

import torch
import time

# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")
model.to(device)

# Função para treinar o modelo com métricas
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        start_time = time.time()

        # Métricas de treino
        train_iou, train_dice, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calcular métricas para o batch
            train_iou += iou(outputs, masks).item()
            train_dice += dice_coefficient(outputs, masks).item()
            train_precision += precision(outputs, masks).item()
            train_recall += recall(outputs, masks).item()
            train_f1 += f1_score(outputs, masks).item()

        # Validação
        model.eval()
        val_loss = 0
        val_iou, val_dice, val_precision, val_recall, val_f1 = 0, 0, 0, 0, 0

        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, masks).item()

                # Calcular métricas para o batch
                val_iou += iou(outputs, masks).item()
                val_dice += dice_coefficient(outputs, masks).item()
                val_precision += precision(outputs, masks).item()
                val_recall += recall(outputs, masks).item()
                val_f1 += f1_score(outputs, masks).item()

        # Exibir progresso
        print(f"📌 Época [{epoch+1}/{epochs}] - Loss Treino: {epoch_loss/len(train_loader):.4f} | Loss Validação: {val_loss/len(val_loader):.4f}")
        print(f"   Métricas de Treino -> IoU: {train_iou/len(train_loader):.4f} | Dice: {train_dice/len(train_loader):.4f} | Prec: {train_precision/len(train_loader):.4f} | Rec: {train_recall/len(train_loader):.4f} | F1: {train_f1/len(train_loader):.4f}")
        print(f"   Métricas de Validação -> IoU: {val_iou/len(val_loader):.4f} | Dice: {val_dice/len(val_loader):.4f} | Prec: {val_precision/len(val_loader):.4f} | Rec: {val_recall/len(val_loader):.4f} | F1: {val_f1/len(val_loader):.4f}")
        print(f"   Tempo da Época: {time.time() - start_time:.2f}s")

# Treinar o modelo com métricas
train_model(model, train_loader, val_loader, epochs=50)


📌 Época [1/50] - Loss Treino: 0.3641 | Loss Validação: 0.3314
   Métricas de Treino -> IoU: 0.5793 | Dice: 0.7247 | Prec: 0.7203 | Rec: 0.7620 | F1: 0.7247
   Métricas de Validação -> IoU: 0.6192 | Dice: 0.7612 | Prec: 0.7380 | Rec: 0.8201 | F1: 0.7612
   Tempo da Época: 2339.66s
📌 Época [2/50] - Loss Treino: 0.2636 | Loss Validação: 0.3038
   Métricas de Treino -> IoU: 0.6684 | Dice: 0.7960 | Prec: 0.8121 | Rec: 0.8070 | F1: 0.7960
   Métricas de Validação -> IoU: 0.6338 | Dice: 0.7679 | Prec: 0.8271 | Rec: 0.7432 | F1: 0.7679
   Tempo da Época: 2359.86s
📌 Época [3/50] - Loss Treino: 0.2181 | Loss Validação: 0.2393
   Métricas de Treino -> IoU: 0.7171 | Dice: 0.8309 | Prec: 0.8498 | Rec: 0.8290 | F1: 0.8309
   Métricas de Validação -> IoU: 0.7126 | Dice: 0.8259 | Prec: 0.8936 | Rec: 0.7869 | F1: 0.8259
   Tempo da Época: 2367.20s
📌 Época [4/50] - Loss Treino: 0.2089 | Loss Validação: 0.2260
   Métricas de Treino -> IoU: 0.7206 | Dice: 0.8330 | Prec: 0.8538 | Rec: 0.8332 | F1: 0.8330
 